# DoLa: Decoding by Contrasting Layers Improves Factuality in Large Language Models

Aunque a medida que los LLMs van aumentando de tamaño y van surgiendo nuevas capacidades, tenemos un problema y son las alucionaciones. Los autores del paper [DoLa: Decoding by Contrasting Layers Improves Factuality in Large Language Models](https://arxiv.org/abs/2309.03883) proponen un método para evitar este problema.

Proponen un enfoque de decodificación contrastiva, donde la probabilidad de salida de la siguiente palabra se obtiene de la diferencia en los logits entre una capa superior y una inferior. Al enfatizar el conocimiento de las capas superiores y restarle importancia al de las inferiores, podemos hacer que los LM sean más factuales y, por lo tanto, reducir las alucinaciones.

En la siguiente figura se muestra esta idea. Mientras que `Seattle` mantiene una alta probabilidad en todas las capas, la probabilidad de la respuesta correcta `Olympia` aumenta después de que las capas superiores inyecten más conocimiento factual. Contrastar las diferencias entre las distintas capas puede revelar la respuesta correcta en este caso

![DoLa-figure1](https://maximofn.com/wp-content/uploads/2024/07/DoLa-figure1.webp)

## Método

Un LLM consiste en una capa de embedding, varios transformers secuenciales y a continuación una capa de salida. Lo que proponen es medir la salida de cada transformer mediante la divergencia de Jensen-Shannon (JSD)

En la siguiente figura se puede ver esta medida a la salida de cada transformer para una frase de entrada al LLM. Cada columna corresponde a un token de la frase

![DoLa-figure2](https://maximofn.com/wp-content/uploads/2024/07/DoLa-figure2.webp)

Se pueden observar dos patrones

 * El primero ocurre cuando se predicen entidades de nombre o fechas importantes, como `Wole Soyinka` y `1986`, que requieren conocimiento factual. Se puede ver que la JSD calculada sigue siendo extremadamente alta en las capas superiores. Este patrón indica que el modelo sigue cambiando sus predicciones en las últimas capas, y potencialmente inyectando más conocimiento factual en las predicciones
 
 * El segundo ocurre cuando se predicen palabras funcionales, como `was`, `the`, `to`, `in`, y los tokens copiados de la pregunta de entrada, como `first Nigerian`, `Nobel Prize`. Cuando se predicen estos tokens "fáciles", podemos observar que la JSD se vuelve muy pequeña a partir de las capas intermedias. Este hallazgo indica que el modelo ya ha decidido qué token generar en las capas intermedias, y mantiene las distribuciones de salida casi sin cambios en las capas superiores. Este hallazgo también es consistente con las suposiciones en los LLM de salida temprana `Schuster et al., 2022`

Cuando la predicción de la siguiente palabra requiere conocimiento factual, el LLM parece cambiar las predicciones en las capas superiores. Contrastar las capas antes y después de un cambio repentino puede, por lo tanto, amplificar el conocimiento que emerge de las capas superiores y hacer que el modelo se base más en su conocimiento interno factual. Además, esta evolución de la información parece variar de un token a otro

Su método requiere seleccionar con precisión la capa prematura que contiene información plausible pero menos factual, que puede no estar siempre en la misma capa temprana. Por lo tanto, proponen encontrar esa capa prematura mediante una selección dinámica de la capa prematura como se ve en la siguiente imagen

![DoLa-figure3](https://maximofn.com/wp-content/uploads/2024/07/DoLa-figure3.webp)

## Selección dinámica de la capa prematura

Para seleccionar la capa prematura calculan la divergencia de Jensen-Shannon (JSD) entre las capas intermedias con la final. La capa prematura se selecciona como la capa con la JSD más alta

Sin embargo, como este proceso puede ser un poco lento, lo que hacen es agrupar varias capas para hacer menos cálculos

## Contraste de las predicciones

Ahora que tenemos la última capa (capa madura) y la capa prematura, podemos contrastar las predicciones de ambas capas. Para ello, calculan la probabilidad logarítmica del siguiente token en la capa madura y la prematura. A continuación restan la probabilidad logarítmica de la capa prematura a la de la capa madura, así le dan más importancia al conocimiento de la capa madura

## Penalización por repetición

La motivación de DoLa es restar importancia al conocimiento lingüístico de las capas inferiores y amplificar el conocimiento factual del mundo real. Sin embargo, esto puede dar lugar a que el modelo genere párrafos gramaticalmente incorrectos

Empíricamente no han observado ese problema, pero han encontrado que la distribución DoLa resultante a veces tiene una mayor tendencia a repetir frases generadas previamente, especialmente durante la generación de largas secuencias de razonamiento en la cadena de pensamiento

Así que incluyen una penalización por repetición introducida en `Keskar et al. (2019)` con `θ = 1.2` durante la decodificación

## Implementación con transformers

Vamos a ver cómo implementar DoLa con la librería `transformers` de Hugging Face. Para obtener más información de cómo aplicar DoLa con la librer´ía `transformers` puedes consultar el siguiente [enlace](https://huggingface.co/docs/transformers/main/en/generation_strategies#dola-decoding)

Primero instanciamos el tokenizador y el modelo

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
checkpoints = "huggyllama/llama-7b"
# checkpoints = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)
model = AutoModelForCausalLM.from_pretrained(checkpoints, torch_dtype=torch.float16)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Asignamos un valor fijo de semilla para la reproducibilidad del ejemplo

In [ ]:
set_seed(42)

Generamos los tokens de entrada al LLM

In [ ]:
text = "On what date was the Declaration of Independence officially signed?"
inputs = tokenizer(text, return_tensors="pt").to(device)

Generamos ahora la entrada vanilla, es decir, sin aplicar DoLa

In [ ]:
vanilla_output = model.generate(**inputs, do_sample=False, max_new_tokens=50)
tokenizer.batch_decode(vanilla_output[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)

Ahora aplicando DoLa

In [ ]:
dola_high_output = model.generate(**inputs, do_sample=False, max_new_tokens=50, dola_layers='high', repetition_penalty=1.2)
tokenizer.batch_decode(dola_high_output[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)

Y ahora una versión personalizada de DoLa, en que solo se contrasta la capa final con las capas 28 y 30

In [ ]:
dola_custom_output = model.generate(**inputs, do_sample=False, max_new_tokens=50, dola_layers=[28,30], repetition_penalty=1.2)
tokenizer.batch_decode(dola_custom_output[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)